Goal of this notebook:
    - Benchmark both ways of determining connections between vertices at a time:
        - Create a method to benchmark both ways by entering a number of vertices and edges, setting up connections, and then timing the queries on a range of times and iterations.

In [ ]:
# Jupyter notebook needs this or else it will crash
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

# Get the vertices of the graph as a list, and print them.

print(g.V().toList())

# METHOD 1

In [ ]:
def connected_1(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>. 
    Do so by sending a Gremlin query to determine whether there exists an edge between the 
    two vertices such that <time> falls in between their "start_time" and "end_time" parameters.
    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # Get the vertices associated with the names
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Return whether there are edges that:
    #   - connect v1 and v2, 
    #   - labelled 'connection',
    #   - have a start time that is less than or equal to <time>
    #   - either do not have an end time or have an end time that is greater than or equal to <time>
    return g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
            __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
            __.or_(
                __.hasNot('end'),           # end time doesn't have to exist 
                __.has('end', P.gt(time))  # OR end time must be greater than <time>
            )
        ).count().next() > 0

In [ ]:
def set_connection_1(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, create a new connection or terminate their existing connection, based on the value of <bool>. Label with time <time>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    if connection:
        if not connected_1(name1, name2, time):
            # Get the vertices associated with the names
            v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

            # Add an edge labelled 'connection' with a start time of <time>
            g.V(v1).addE('connection').from_(v1).to(v2).property('start', time).iterate()

    else:

        # Get the vertices associated with the names
        v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

        # For all edges between v1 and v2 labelled 'connection' (there should only be one) that do not have an 'end' property, create an end property of <time>.
        g.V(v1).bothE('connection').filter(__.bothV().is_(v2)).hasNot('end').property('end', time).iterate()

# METHOD 2

In [ ]:
def get_next_smallest_index(val, lst) -> int:
    """
    Given a sorted list lst in increasing order and val where val is of the same type as all elements in lst, 
    do a binary search and return the lower bound on the index (if the exact value is not found).
    """

    l, r = 0, len(lst) - 1

    while l <= r:
        mid = l + (r - l) // 2 
        if val > lst[mid]:
            l = mid + 1
        elif val < lst[mid]:
            r = mid - 1
        else:
            return mid
    return l - 1

# lst = [1, 3, 5, 7]
# for i in range(10):
#     print(i, lst[get_next_smallest_index(i, lst)])


In [ ]:
def connected_2(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>.
    Do so by sending a Gremlin query to find an edge labelled 'connected' between the two vertices. If one exists, get its "log" of changes to connections (which is just a list), and do a binary search to determine whether at <time>, the edge was active.
    """

    # Get the vertices associated with the names

    # REMOVE THIS LINE: it is inefficient, just find the vertices in the same query as getting the list.
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Get the "connection log" list of the edge that connects v1 and v2. If no edge exists, this should be an empty list.
    # The list should be of the format [[t, c], [t, c], ..., [t, c]], where t is the time of the change and c represents whether
    # the element was connected or disconnected at the time.

    # connections_list = g.V(v1).bothE('connection').filter(__.bothV().is_(v2)).values('connection_log').toList()

    connections_list = g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').values('connection_log').toList()

    if len(connections_list) == 0:
        return False
    else:
        connections = connections_list[0]

        # How long each segment of the string is (1 for C, 10 for the Ts)
        n = 11

        # Get a list of the times of the connections
        times = [int(connections[i + 1: i + n]) for i in range(0, len(connections), n)]

        index = get_next_smallest_index(time, times)

        # print(connections, index)

        if index == -1:
            return False
        else:
            return bool(int(connections[index * n]))

In [ ]:
def set_connection_2(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, either add a new entry to the connection log of the 'connection' edge connecting the two vertices, or create a new edge if such an edge does not already exist.
    
    Preconditions:
         - time is greater than the times of all previous connections!

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # This prevents multiple entries of the same type.
    if connection != connected_2(name1, name2, time):

        # Get the vertices associated with the names
        v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

        # this is either an empty list or a list containing one string
        entry_list = g.V(v1).bothE('connection').filter(__.bothV().is_(v2)).values('connection_log').toList()

        if len(entry_list) == 0:
            prev_entry = ""
        else:
            prev_entry = entry_list[0]

        # Append CTTTTTTTTTT to the previous log entry
        entry = prev_entry + str(int(connection)) + str(int(time)).zfill(10)

        g.V(v1).bothE('connection').filter(__.bothV().is_(v2)).fold().coalesce(
            __.unfold(), 
            __.addE('connection').from_(v1).to(v2)
        ).property('connection_log', entry).iterate()
        

# BENCHMARK SETUP

In [ ]:
def clear_graph() -> None:
    """
    Drop the vertices and edges of the graph.
    """
    g.V().drop().iterate()
    g.E().drop().iterate()

In [ ]:
def populate_vertices(vertices: list) -> None:
    """
    For each element of <vertices>, add a vertex to the graph with that name.
    """

    for name in vertices:
        g.addV().property('name', name).iterate()

In [ ]:
def populate_connections_1(connections: list) -> None:
    """
    Using method 1, populate the connections of the vertices in the graph.

    connections should be a list containing tuples of the type
    ((name1, name2), (time, connected)) where name1 and name2 are names of existing vertices in the graph, and the list is sorted by time in increasing order.
    """

    for ((name1, name2), (time, connected)) in connections:
        set_connection_1(name1=name1, name2=name2, time=time, connection=connected)

In [ ]:
def setup_method_1(vertices: list, connections: list) -> None:
    """
    Using method 1, clear the graph, populate the vertices, and populate the connections.
    """

    clear_graph()
    populate_vertices(vertices)
    populate_connections_1(connections)

In [ ]:
def populate_connections_2(connections: list) -> None:
    """
    Using method 2, populate the connections of the vertices in the graph.

    connections should be a list containing tuples of the type
    ((name1, name2), (time, connected)) where name1 and name2 are names of existing vertices in the graph, and the list is sorted by time in increasing order.
    """

    for ((name1, name2), (time, connected)) in connections:
        set_connection_2(name1=name1, name2=name2, time=time, connection=connected)

In [ ]:
def setup_method_2(vertices: list, connections: list) -> None:
    """
    Using method 2, clear the graph, populate the vertices, and populate the connections.
    """

    clear_graph()
    populate_vertices(vertices)
    populate_connections_2(connections)

# BENCHMARKS

In [ ]:
import itertools
import random
import datetime

def benchmark(vertices_count: int, connections_count: int, calls_count: int) -> any:
    """
    Pregenerate lists of connections between the vertices, then test both methods using these lists.

    Return the setup time, and query time for each of the methods in a tuple
    """

    # print(f"\n\nBenchmarking both methods with {vertices_count} vertices, {connections_count} connections, and {calls_count} calls")

    vertices = ["v" + str(i) for i in range(vertices_count)]

    combinations = list(itertools.combinations(vertices, 2))

    # Remember what the last connection status was for each combination of vertices.
    last_connections = {comb: (0, False) for comb in combinations}

    connections = []

    time_min = -1
    time_max = 0

    for _ in range(connections_count):
        
        # Combination 
        comb = random.choice(combinations)

        # Last connection for this combination
        last = last_connections[comb]

        # Times will be in intervals of 2 (arbitrary)
        time = last[0] + 2

        if time > time_max:
            time_max = time

        # Alternate between connected and disconnected
        connected = not last[1]

        connection = (time, connected)

        last_connections[comb] = connection

        connections.append((comb, connection))

    calls = [(random.choice(combinations), random.randint(time_min, time_max + 1)) for _ in range(calls_count)]

    # Test method 1

    # print("\n--------")
    # print("METHOD 1")
    # print("--------")

    # Setup
    now = datetime.datetime.now()

    setup_method_1(vertices=vertices, connections=connections)

    setup_time_1 = (datetime.datetime.now() - now).total_seconds()

    # print("Set up time:", setup_time_1)


    # Queries
    now = datetime.datetime.now()

    for ((name1, name2), time) in calls:
        connected_1(name1, name2, time)

    query_time_1 = (datetime.datetime.now() - now).total_seconds()

    average = query_time_1 / calls_count

    
    # print("Total time:", query_time_1)
    # print("Average time:", average)



    # Test method 2

    # print("\n--------")
    # print("METHOD 2")
    # print("--------")

    # Setup
    now = datetime.datetime.now()

    setup_method_2(vertices=vertices, connections=connections)

    setup_time_2 = (datetime.datetime.now() - now).total_seconds()

    # print("Set up time:", setup_time_2)


    # Queries
    now = datetime.datetime.now()

    for ((name1, name2), time) in calls:
        connected_2(name1, name2, time)

    query_time_2 = (datetime.datetime.now() - now).total_seconds()

    average = query_time_2 / calls_count

    
    # print("Total time:", query_time_2)
    # print("Average time:", average)

    return ((setup_time_1, query_time_1), (setup_time_2, query_time_2))



In [ ]:
benchmark(vertices_count=10, connections_count=10, calls_count=10)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot(vertices_count: int, connections_count: int, calls_list: list) -> None:
    """
    For each calls count in calls_list, run benchmark and then plot the results.
    """

    setup_times_1, setup_times_2, query_times_1, query_times_2 = [], [], [], []

    for calls_count in calls_list:
        ((setup_time_1, query_time_1), (setup_time_2, query_time_2)) = benchmark(
                                                                                vertices_count=vertices_count, 
                                                                                connections_count=connections_count, 
                                                                                calls_count=calls_count)
        setup_times_1.append(setup_time_1)
        setup_times_2.append(setup_time_2)
        query_times_1.append(query_time_1 / calls_count)
        query_times_2.append(query_time_2 / calls_count)

    plt.plot(calls_list, query_times_1, label="Method 1 Query Times")
    plt.plot(calls_list, query_times_2, label="Method 2 Query Times")
    plt.title(f"{vertices_count} vertices, {connections_count} connections")
    plt.xlabel("Number of Calls")
    plt.ylabel("Average Query Time (s)")
    plt.legend()
    plt.show()



In [ ]:
plot(vertices_count=10, connections_count=10, calls_list=list(range(10, 110, 10)))